In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import codecs
import pandas as pd
with codecs.open('ratings_test.txt', encoding='utf-8') as f:
    data=[i.split('\t') for i in f.read().splitlines()]
    m=data[0] #맨 첫번째는 메뉴임
    data=data[1:]

In [2]:
p_data = pd.DataFrame(data, columns = m)
p_data

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [9]:
p_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [5]:
X_data = p_data['document']
y_data = p_data['label']

In [8]:
X_data.shape, y_data.shape

((50000,), (50000,))

In [10]:
import numpy as np
np.unique(y_data) #이진분류

array(['0', '1'], dtype=object)

1. 토큰화

In [3]:
! pip install kss konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.4 MB 1.3 MB/s 
     |████████████████████████████████| 19.4 MB 10.3 MB/s 
     |████████████████████████████████| 131 kB 44.7 MB/s 
     |████████████████████████████████| 453 kB 29.1 MB/s 
  Created wheel for kss: filename=kss-3.4.3-py3-none-any.whl size=42448068 sha256=999636dbae0dc155e98a5cf044b846aea7cbccfacf90b196e1980f33d2535d6a
  Stored in directory: /root/.cache/pip/wheels/10/df/7d/0d0a58843aa7d267687671cf57274080bcfe8f79d59ed8f399
Successfully built kss


In [48]:
from konlpy.tag import Okt
from tqdm import tqdm
tk = Okt()
tk_x = []
for i in tqdm(X_data):
    tk_x.append(tk.morphs(i, stem = True))


100%|██████████| 50000/50000 [04:03<00:00, 204.99it/s]


In [52]:
len(np.unique(tk_x))

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


48875

In [55]:
n_l = []
x = np.unique(tk_x)
for i in x:
    n = len(i)
    n_l.append(n)
sum(n_l) #?????

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


719618

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tkn = Tokenizer(num_words=5000)
tkn.fit_on_texts(tk_x)
tkn.word_index

In [56]:
x_data1 = tkn.texts_to_matrix(tk_x) #단어유무
x_data2 = tkn.texts_to_matrix(tk_x, mode='count') #단어 빈도수
x_data3 = tkn.texts_to_matrix(tk_x, mode='tfidf') #tfidf

In [57]:
x_data3.shape

(50000, 5000)

2. 데이터 전처리

In [58]:
Y_data = to_categorical(y_data)

In [59]:
Y_data.shape

(50000, 2)

In [24]:
t_x1, tt_x1, t_y1, tt_y1 = train_test_split(x_data1, Y_data, random_state = 1)
t_x2, tt_x2, t_y2, tt_y2 = train_test_split(x_data2, Y_data, random_state = 1)
t_x3, tt_x3, t_y3, tt_y3 = train_test_split(x_data3, Y_data, random_state = 1)

In [35]:
t_x1.shape, t_y1.shape

((37500, 100), (37500, 2))

In [36]:
type(t_x1), type(t_y1)

(numpy.ndarray, numpy.ndarray)

3. 모델 생성 및 학습

In [60]:
del m

In [38]:
m = Sequential()
m.add(Dense(2, activation= 'softmax',input_shape=(5000,))) #결과값이 두개로 나오니까 softmax 써야한다.
m.compile(optimizer= 'adam',loss='categorical_crossentropy', metrics = 'accuracy')

In [39]:
hy = m.fit(t_x1, t_y1, validation_data=['tt_x1','tt_y1'], epochs = 5)

Epoch 1/5
1171/1172 [============================>.] - ETA: 0s - loss: 0.6442 - accuracy: 0.6178

IndexError: ignored

4. 그래프 표현